### Feature Selection 

In [1]:
import pandas as pd
import numpy as np
import os

import sklearn

import xgboost as xgb

from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold

import logging

In [5]:
# Pandas configuration 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
LOGGER = logging.getLogger(__name__)

In [7]:
path_to_data = os.getenv('Documents') + '/House Prices - Advanced Regression Techniques/Datasets/'

In [8]:
train_data = pd.read_csv(path_to_data + 'train_clean.csv')

In [9]:
train_data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_NA,GarageQual_Po,GarageQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NA,BsmtCond_Po,BsmtCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,PavedDrive_N,PavedDrive_P,PavedDrive_Y,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NA,BsmtQual_TA,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NA,BsmtFinType1_Rec,BsmtFinType1_Unf,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NA,FireplaceQu_Po,FireplaceQu_TA,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NA,BsmtExposure_No,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_NA,GarageCond_Po,GarageCond_TA,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,CentralAir_N,CentralAir_Y,SaleType_COD,SaleType_CWD,SaleType_C

In [10]:
y = train_data['SalePrice']
X = train_data.drop(columns=['SalePrice'], axis=1)

In [11]:
features = X.columns
features_np = np.asarray(features)

In [12]:
features_np

array(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold',
       'GarageQual_Ex', 'GarageQual_Fa', 'GarageQual_Gd', 'GarageQual_NA',
       'GarageQual_Po', 'GarageQual_TA', 'BsmtCond_Fa', 'BsmtCond_Gd',
       'BsmtCond_NA', 'BsmtCond_Po', 'BsmtCond_TA', 'Foundation_BrkTil',
       'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab',
       'Foundation_Stone', 'Foundation_Wood', 'LandContour_Bnk',
       'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl',
       'Neighborhood_Blmngtn', 'Neighborhood_Blueste',
      

In [13]:
# X contains 283 columns

In [14]:
train_data['LotFrontage'] = pd.to_numeric(train_data['LotFrontage'])

### Feature Selection

In [15]:
# Conservative parameters
# TO DO ... look into different parameters?

params = {
    'n_estimators':10,
    'max_depth':5
}

In [16]:
# Using XGBoost sklearn API to thus access sklearn functions

model = xgb.XGBRegressor()

In [17]:
# With step = 3

min_features_to_select = 1

rfecv = RFECV(estimator=model,
           step=3,
           min_features_to_select=1,
           cv=StratifiedKFold(3), 
           scoring='neg_mean_absolute_error',
           verbose=3
        )

In [18]:
results_stp3 = rfecv.fit(X, y)

C:\Users\naylora1\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


Fitting estimator with 283 features.
Fitting estimator with 280 features.
Fitting estimator with 277 features.
Fitting estimator with 274 features.
Fitting estimator with 271 features.
Fitting estimator with 268 features.
Fitting estimator with 265 features.
Fitting estimator with 262 features.
Fitting estimator with 259 features.
Fitting estimator with 256 features.
Fitting estimator with 253 features.
Fitting estimator with 250 features.
Fitting estimator with 247 features.
Fitting estimator with 244 features.
Fitting estimator with 241 features.
Fitting estimator with 238 features.
Fitting estimator with 235 features.
Fitting estimator with 232 features.
Fitting estimator with 229 features.
Fitting estimator with 226 features.
Fitting estimator with 223 features.
Fitting estimator with 220 features.
Fitting estimator with 217 features.
Fitting estimator with 214 features.
Fitting estimator with 211 features.
Fitting estimator with 208 features.
Fitting estimator with 205 features.
F

Fitting estimator with 175 features.
Fitting estimator with 172 features.
Fitting estimator with 169 features.
Fitting estimator with 166 features.
Fitting estimator with 163 features.
Fitting estimator with 160 features.
Fitting estimator with 157 features.
Fitting estimator with 154 features.
Fitting estimator with 151 features.
Fitting estimator with 148 features.
Fitting estimator with 145 features.
Fitting estimator with 142 features.
Fitting estimator with 139 features.
Fitting estimator with 136 features.
Fitting estimator with 133 features.
Fitting estimator with 130 features.
Fitting estimator with 127 features.
Fitting estimator with 124 features.
Fitting estimator with 121 features.
Fitting estimator with 118 features.
Fitting estimator with 115 features.
Fitting estimator with 112 features.
Fitting estimator with 109 features.
Fitting estimator with 106 features.
Fitting estimator with 103 features.
Fitting estimator with 100 features.
Fitting estimator with 97 features.
Fi

In [19]:
# Number of selected features

results_stp3.n_features_

106

In [20]:
# Analysing results from 3 step RFECV

results_df3 = pd.DataFrame(columns=['Ranking 3 step'], index=[features_np])
results_df3['Ranking 3 step'] = results_stp3.ranking_

In [21]:
results_df3.sort_values(by=['Ranking 3 step'])

,Ranking 3 step
MSZoning_RM,1
Condition2_Feedr,1
RoofMatl_CompShg,1
KitchenQual_Ex,1
Neighborhood_Timber,1
Neighborhood_StoneBr,1
Neighborhood_Somerst,1
KitchenQual_Gd,1
KitchenQual_TA,1
Neighborhood_OldTown,1


In [22]:
# Using the selected feature mask to return the selected features from RFECV

sel_features_stp3 = features_np[results_st3.support_].tolist()

In [23]:
sel_features_stp3

['LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'BsmtFullBath',
 'FullBath',
 'BedroomAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'GarageQual_Fa',
 'GarageQual_TA',
 'BsmtCond_TA',
 'Foundation_CBlock',
 'Foundation_PConc',
 'LandContour_Bnk',
 'LandContour_HLS',
 'LandContour_Lvl',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_NAmes',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'RoofMatl_CompShg',
 'GarageFinish_RFn',
 'GarageFinish_Unf',
 'Functional_Maj2',
 'Functional_Typ',
 'LandSlope_Gtl',
 'LandSlop

In [26]:
# Now with 1 step to compare results

model = xgb.XGBRegressor()

rfecv = RFECV(estimator=model,
           step=1,
           min_features_to_select=1,
           cv=StratifiedKFold(3), 
           scoring='neg_mean_absolute_error',
           verbose=3
        )

In [28]:
results_stp1 = rfecv.fit(X, y)

C:\Users\naylora1\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


Fitting estimator with 283 features.
Fitting estimator with 282 features.
Fitting estimator with 281 features.
Fitting estimator with 280 features.
Fitting estimator with 279 features.
Fitting estimator with 278 features.
Fitting estimator with 277 features.
Fitting estimator with 276 features.
Fitting estimator with 275 features.
Fitting estimator with 274 features.
Fitting estimator with 273 features.
Fitting estimator with 272 features.
Fitting estimator with 271 features.
Fitting estimator with 270 features.
Fitting estimator with 269 features.
Fitting estimator with 268 features.
Fitting estimator with 267 features.
Fitting estimator with 266 features.
Fitting estimator with 265 features.
Fitting estimator with 264 features.
Fitting estimator with 263 features.
Fitting estimator with 262 features.
Fitting estimator with 261 features.
Fitting estimator with 260 features.
Fitting estimator with 259 features.
Fitting estimator with 258 features.
Fitting estimator with 257 features.
F

Fitting estimator with 60 features.
Fitting estimator with 59 features.
Fitting estimator with 58 features.
Fitting estimator with 57 features.
Fitting estimator with 56 features.
Fitting estimator with 55 features.
Fitting estimator with 54 features.
Fitting estimator with 53 features.
Fitting estimator with 52 features.
Fitting estimator with 51 features.
Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 fe

Fitting estimator with 118 features.
Fitting estimator with 117 features.
Fitting estimator with 116 features.
Fitting estimator with 115 features.
Fitting estimator with 114 features.
Fitting estimator with 113 features.
Fitting estimator with 112 features.
Fitting estimator with 111 features.
Fitting estimator with 110 features.
Fitting estimator with 109 features.
Fitting estimator with 108 features.
Fitting estimator with 107 features.
Fitting estimator with 106 features.
Fitting estimator with 105 features.
Fitting estimator with 104 features.
Fitting estimator with 103 features.
Fitting estimator with 102 features.
Fitting estimator with 101 features.
Fitting estimator with 100 features.
Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
Fitting estimator with 92 features.
Fitting e

Fitting estimator with 175 features.
Fitting estimator with 174 features.
Fitting estimator with 173 features.
Fitting estimator with 172 features.
Fitting estimator with 171 features.
Fitting estimator with 170 features.
Fitting estimator with 169 features.
Fitting estimator with 168 features.
Fitting estimator with 167 features.
Fitting estimator with 166 features.
Fitting estimator with 165 features.
Fitting estimator with 164 features.
Fitting estimator with 163 features.
Fitting estimator with 162 features.
Fitting estimator with 161 features.
Fitting estimator with 160 features.
Fitting estimator with 159 features.
Fitting estimator with 158 features.
Fitting estimator with 157 features.
Fitting estimator with 156 features.
Fitting estimator with 155 features.
Fitting estimator with 154 features.
Fitting estimator with 153 features.
Fitting estimator with 152 features.
Fitting estimator with 151 features.
Fitting estimator with 150 features.
Fitting estimator with 149 features.
F

Fitting estimator with 232 features.
Fitting estimator with 231 features.
Fitting estimator with 230 features.
Fitting estimator with 229 features.
Fitting estimator with 228 features.
Fitting estimator with 227 features.
Fitting estimator with 226 features.
Fitting estimator with 225 features.
Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
Fitting estimator with 209 features.
Fitting estimator with 208 features.
Fitting estimator with 207 features.
Fitting estimator with 206 features.
F

In [29]:
# Number of features selected

results_stp1.n_features_

94

In [32]:
# Analysing results from 1 step RFECV

results_df1 = pd.DataFrame(columns=['Ranking 1 step'], index=[features_np])
results_df1['Ranking 1 step'] = results_stp1.ranking_

In [33]:
results_df1.sort_values(by=['Ranking 1 step'])

,Ranking 1 step
MSZoning_RM,1
Neighborhood_Timber,1
Neighborhood_StoneBr,1
Neighborhood_Somerst,1
KitchenQual_Gd,1
KitchenQual_TA,1
Neighborhood_OldTown,1
HouseStyle_1.5Fin,1
HouseStyle_SLvl,1
Neighborhood_NAmes,1


In [35]:
# Using the selected feature mask to return the selected features from RFECV

sel_features_stp1 = features_np[results_stp1.support_].tolist()

In [37]:
sel_features_stp1

['LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'FullBath',
 'HalfBath',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'PoolArea',
 'MoSold',
 'YrSold',
 'GarageQual_Fa',
 'GarageQual_TA',
 'BsmtCond_Fa',
 'BsmtCond_TA',
 'Foundation_CBlock',
 'LandContour_Bnk',
 'LandContour_HLS',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_NAmes',
 'Neighborhood_OldTown',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'RoofMatl_CompShg',
 'GarageFinish_Unf',
 'Functional_Maj2',
 'Functional_Typ',
 'LandSlope_Gtl',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_HdBoard',
 'Exterior2nd_Wd Sdng',
 'Exterior1st_BrkFace',

In [38]:
# Difference between step 3 and step 1

set(sel_features_stp3).difference(set(sel_features_stp1))

{'BedroomAbvGr',
 'BsmtFinSF2',
 'Electrical_SBrkr',
 'Exterior2nd_AsbShng',
 'FireplaceQu_Fa',
 'Foundation_PConc',
 'GarageFinish_RFn',
 'HouseStyle_1Story',
 'LandContour_Lvl',
 'LandSlope_Mod',
 'LotConfig_Inside',
 'LotShape_IR3',
 'MasVnrType_BrkFace',
 'MiscVal',
 'Neighborhood_NridgHt',
 'RoofStyle_Hip',
 'SaleCondition_Normal'}

In [43]:
# Exporting results

train_data_stp3 = train_data[sel_features_stp3]
train_data_stp1 = train_data[sel_features_stp1]

In [45]:
train_data_stp3['SalePrice'] = y
train_data_stp1['SalePrice'] = y

C:\Users\naylora1\AppData\Local\Temp/ipykernel_7924/2301456437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_stp3['SalePrice'] = y
C:\Users\naylora1\AppData\Local\Temp/ipykernel_7924/2301456437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_stp1['SalePrice'] = y


In [47]:
train_data_stp3.to_csv(path_to_data + 'train_clean_stp3.csv', index=False)
train_data_stp1.to_csv(path_to_data + 'train_clean_stp1.csv', index=False)